In [1]:
import json
import os
import sys
from collections import Counter, defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

src_dir = os.path.abspath(os.path.join(os.pardir, os.pardir,'src'))
sys.path[0] = src_dir
print(sys.path[0])
from parser.support import ROLES, CREDITS
from parser.my_mongo_db_login import DB_LOGIN_INFO
import parser.support as support
import network.shift_graph_maker as sgm
import model.contagion as contagion

/home/staff/junelee/Research-Project/movie_propagation_reserve/src


In [2]:
%reload_ext autoreload
%autoreload 2
%load_ext snakeviz

%matplotlib inline

In [3]:
with open('/home/projects/movie-network/data/raw_data/movies.json') as f:
    movie_file = f.read()
    movie_data = json.loads(movie_file)

In [4]:
role = 'producing'
role_key = role + "_gender_percentage"
all_movies = support.get_movies_df(role_key)
print('Got all_movies')
gender_df = support.get_staff_df('producers')[['_id', 'female_count', 'first_movie', 'last_movie', 'gender']]

Loaded IMDb movies producing_gender_percentage
Got all_movies


In [5]:
movie_90s = all_movies[(all_movies.year >= 1990) & (all_movies.year < 2000)]
movie_90s = movie_90s.sort_values('year')

In [13]:
len(movie_90s)

2009

In [6]:
all_movies.columns
#get 90s movies with only producer, id, title, year...
movie_producer_df = movie_90s[['_id', 'producers', 'producing_gender_percentage', 'title', 'year']]

In [7]:
producers = gender_df._id.tolist()

In [8]:
gender_df = support.get_staff_df('producers')[['_id', 'female_count', 'first_movie', 'last_movie', 'gender']]

In [76]:
seeds = sgm.generate_gender_seeds(gender_df)
G = sgm.build_temporal_network(movie_90s, seeds, 'empirical', 1.0)

In [77]:
producers = [n for n in G.nodes() if G.node[n]['node_type']=='P']
len([n for n in producers if G.node[n]['status']=='Adopter'])

1064

In [59]:
#women
counter = 0
for n in G.nodes():
    if G.node[n]['node_type'] == 'P':
        if G.node[n]['gender'] == 1:
            counter += 1
print(counter)

1064


In [82]:
G.node[producers[2]]

{'node_type': 'P',
 'belief': 1.0,
 'status': 'Adopter',
 'movies': ['tt0099622'],
 'roles': [['associate producer', 'tt0099622', 1990]],
 'gender': 1}

In [60]:
movie_nodes = [n for n in G.nodes() if G.node[n]['node_type']=='M']

In [63]:
movie_nodes[-1]

2009

### Contagon

In [64]:
p, d, b = 0.1, 1.0, 1.0
adopter_history = contagion.contagion_belief_propagation_temporal_network(G, p,d,b)

In [67]:
len(adopter_history)

2010

In [15]:
len(producers)

3171

In [16]:
from itertools import product